In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


# Prepare

In [5]:
print(os.listdir("../input"))

['home-credit-default-risk']


In [6]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import sklearn
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings("ignore")
application_train=pd.read_csv('../input/home-credit-default-risk/application_train.csv')

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


### Stratified Sampling (ratio=0.1)

In [7]:
application_sample1=application_train.loc[application_train.TARGET==1].sample(frac=0.1,replace=False)
print('label 1 sample size:',str(application_sample1.shape[0]))
application_sample0= application_train.loc[application_train.TARGET==0].sample(frac=0.1,replace=False)
print('lable 0 sample size:',str(application_sample0.shape[0]))
application=pd.concat([application_sample1,application_sample0],axis=0).sort_values('SK_ID_CURR')

label 1 sample size: 2482
lable 0 sample size: 28269


In [8]:
application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
8,100011,0,Cash loans,F,N,Y,0,112500.0,1019610.0,33826.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
34,100040,0,Cash loans,F,N,Y,0,135000.0,1125000.0,32895.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
38,100045,0,Cash loans,F,N,Y,0,99000.0,247275.0,17338.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
40,100047,1,Cash loans,M,N,Y,0,202500.0,1193580.0,35028.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,4.0
53,100061,0,Cash loans,F,N,Y,2,225000.0,314055.0,16164.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


### Impute missing values

In [9]:
categorical_list=[]
numerical_list=[]
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:',str(len(categorical_list)))
print('Number of numerical features:',str(len(numerical_list)))

Number of categorical features: 16
Number of numerical features: 106


In [10]:
from sklearn.impute import SimpleImputer
application[numerical_list]= SimpleImputer(strategy='median').fit_transform(application[numerical_list])

### Deal with Categorical features: OneHotEncoding

In [11]:
del application_train; gc.collect()
application=pd.get_dummies(application,drop_first=True)
print(application.shape)

(30751, 225)


### Feature matrix and target

In [12]:
X=application.drop(['SK_ID_CURR','TARGET'],axis=1)
y=application.TARGET
feature_name=X.columns.tolist()

# Feature Selection
* select 100 features from 226
* xxx_support: list to represent select this feature or not
* xxx_feature: the name of selected features

## 1 Filter
* documentation for SelectKBest...

### 1.1 Pearson Correlation

**Note**
* Normalization: no
* Impute missing values: yes

In [56]:
def cor_selector(X,y):
    cor_list=[]
    #calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor=np.corrcoef(X[i],y)[0,1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list=[0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature=X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select. 1 for select
    cor_support=[True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [57]:
cor_support, cor_feature=cor_selector(X,y)
print(str(len(cor_feature)),'selected features')

100 selected features


### 1.2 Chi-2

**Note**
* Normalization: MinMaxScalter(values should be bigger than 0)
* Impute missing values: yes

In [58]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm=MinMaxScaler().fit_transform(X)
chi_selector=SelectKBest(chi2,k=100)
chi_selector.fit(X_norm,y)

SelectKBest(k=100, score_func=<function chi2 at 0x7f5d698efa70>)

In [59]:
chi_support=chi_selector.get_support()
chi_feature=X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)),'selected features')

100 selected features


## 2. Wrapper
* documentation for **RFE**

**Note**
* Normalization: depend on the used model; yes for LR
* Impute missing values: depend on the used model; yes for LR

In [18]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector=RFE(estimator=LogisticRegression(),n_features_to_select=100,step=10,verbose=5)
rfe_selector.fit(X_norm,y)

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.


RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10,
    verbose=5)

In [19]:
rfe_support=rfe_selector.get_support()
rfe_feature=X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)),'selected features')

100 selected features


## 3. Embeded
### 3.1 Logistic Regression L1

**Note**
* Normalization : yes
* Impute missing values : yes

In [49]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector=SelectFromModel(LogisticRegression(C=1,penalty="l1",solver='liblinear'),'1.25*median')
embeded_lr_selector.fit(X_norm,y)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'),
                threshold='1.25*median')

In [50]:
embeded_lr_support=embeded_lr_selector.get_support()
embeded_lr_feature=X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)),'selected features')

104 selected features


### 3.2 Random Forest

**Note**
* Normalization: NO
* Impute missing values: yes

In [35]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector=SelectFromModel(RandomForestClassifier(n_estimators=100),threshold='1.25*median')
embeded_rf_selector.fit(X,y)

SelectFromModel(estimator=RandomForestClassifier(), threshold='1.25*median')

In [37]:
embeded_rf_support=embeded_rf_selector.get_support()
embeded_rf_feature=X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)),'selected features')

97 selected features


## 3.3 LightGBM

**Note**
* Normalization: no
* Impute missing values: no

In [53]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

import re
X_1 = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

lgbc=LGBMClassifier(n_estimators=500,learning_rate=0.05, num_leaves=32, colsample_bytree=0.2, reg_alpha=3,reg_lambda=1,min_split_gain=0.01,min_child_weight=40)

embeded_lgb_selector=SelectFromModel(lgbc,threshold='1.25*median')
embeded_lgb_selector.fit(X_1,y)

SelectFromModel(estimator=LGBMClassifier(colsample_bytree=0.2,
                                         learning_rate=0.05,
                                         min_child_weight=40,
                                         min_split_gain=0.01, n_estimators=500,
                                         num_leaves=32, reg_alpha=3,
                                         reg_lambda=1),
                threshold='1.25*median')

In [54]:
embeded_lgb_support=embeded_lgb_selector.get_support()
embeded_lgb_feature=X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)),'selected features')

111 selected features


 ## Summary


In [60]:
pd.set_option('display.max_rows',None)
# put all selection together
feature_selection_df=pd.DataFrame({'Feature':feature_name,'Pearson':cor_support,'Chi-2':chi_support,'RFE':rfe_support,'Logistics':embeded_lr_support, 'Random Forest':embeded_rf_support,'LightGBM':embeded_lgb_support})
# count the selected times of each feature
feature_selection_df['Total']=np.sum(feature_selection_df,axis=1)
# display the top 100
feature_selection_df=feature_selection_df.sort_values(['Total','Feature'],ascending=False)
feature_selection_df.index=range(1,len(feature_selection_df)+1)
feature_selection_df.head(100)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,REGION_RATING_CLIENT_W_CITY,True,True,True,True,True,True,6
2,REGION_POPULATION_RELATIVE,True,True,True,True,True,True,6
3,ORGANIZATION_TYPE_Trade: type 7,True,True,True,True,True,True,6
4,ORGANIZATION_TYPE_Self-employed,True,True,True,True,True,True,6
5,ORGANIZATION_TYPE_Business Entity Type 3,True,True,True,True,True,True,6
6,NONLIVINGAREA_MODE,True,True,True,True,True,True,6
7,NAME_FAMILY_STATUS_Single / not married,True,True,True,True,True,True,6
8,NAME_EDUCATION_TYPE_Secondary / secondary special,True,True,True,True,True,True,6
9,FLOORSMAX_MEDI,True,True,True,True,True,True,6
10,FLAG_OWN_CAR_Y,True,True,True,True,True,True,6
